In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install vaderSentiment

In [ ]:
from __future__ import unicode_literals, print_function

import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding
import pandas as pd
from fastprogress import progress_bar
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
def main(LABEL, TRAIN_DATA, new_model_name, output_dir=None, n_iter=10):
    random.seed(0)
    nlp = spacy.blank("en")
    print("Created blank 'en' model")
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner)
    else:
        ner = nlp.get_pipe("ner")

    ner.add_label(LABEL)
    optimizer = nlp.begin_training()
    move_names = list(ner.move_names)
    pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
    with nlp.disable_pipes(*other_pipes): 
        sizes = compounding(1.0, 4.0, 1.001)
        for itn in progress_bar(range(n_iter)):
            random.shuffle(TRAIN_DATA)
            batches = minibatch(TRAIN_DATA, size=sizes)
            losses = {}
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
            print("Losses", losses)
    
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.meta["name"] = new_model_name  # rename model
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

In [ ]:
df_train = pd.read_csv("../input/tweet-sentiment-extraction/train.csv")

In [ ]:
train_pos = df_train[df_train['sentiment'] == 'positive']
train_neg = df_train[df_train['sentiment'] == 'negative']

In [ ]:
def get_data(x):
    s = x['text']
    sub = x['selected_text']
    st = s.find(sub)
    en = st + len(sub)
    return (s, {'entities': [(st, en, 'core')]})

In [ ]:
x = train_pos.apply(get_data, axis=1)
TRAIN_DATA = list(x)
main("core", TRAIN_DATA, "positive", "positive", 10)

In [ ]:
x = train_neg.apply(get_data, axis=1)
TRAIN_DATA = list(x)
main("core", TRAIN_DATA, "negative", "negative", 10)

In [ ]:
test = pd.read_csv("../input/tweet-sentiment-extraction/test.csv")

In [ ]:
def vader(s, sentiment):
    analyzer = SentimentIntensityAnalyzer()
    word_list = s.split()
    max_score = -0.01
    max_word = ""
    for word in word_list:
        if sentiment == 'positive':
            word_score = analyzer.polarity_scores(word)['pos']
        else:
            word_score = analyzer.polarity_scores(word)['neg']
        if word_score > max_score:
            max_word = word
            max_score = word_score
        #print(word, word_score)
    return max_word

In [ ]:
def extract(x):
    if x['sentiment'] == 'neutral':
        return x['text']
    if len(x['text'].lower().split()) <= 3:
        return x['text']
    if x['sentiment'] == 'positive':
        doc  = pos_model(x['text'])
    else:
        doc  = neg_model(x['text'])
    if len(doc.ents) != 0:
        return doc.ents[0].text
    else:
        return vader(x['text'], x['sentiment'])

In [ ]:
pos_model = spacy.load("positive")
neg_model = spacy.load("negative")
model = spacy.load("en_core_web_sm")

In [ ]:
test['selected_text'] = test.apply(extract, axis = 1)

In [ ]:
result = test[['textID', 'selected_text']]

In [ ]:
result.to_csv("submission.csv", index=False)